<a href="https://www.kaggle.com/code/darkurthe/the-izumi-sakai-lyrics-ai-capstone-project?scriptVersionId=232763739" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# ## Step 1: Install necessary libraries
import subprocess

def install_package(package_name):
    try:
        subprocess.check_call(['pip', 'install', package_name, '-q'])
        print(f"Installed {package_name}")
    except subprocess.CalledProcessError as e:
        print(f"Error installing {package_name}: {e}")

install_package('google-generativeai')
install_package('pillow')
install_package('beautifulsoup4')
install_package('google-generativeai')
install_package('google-api-python-client')

# ## Step 2: Import necessary modules
import os
from bs4 import BeautifulSoup
import re
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai
from PIL import Image
import io
import base64
from IPython.display import display
from googleapiclient.discovery import build

# ## Step 3: Define the directory containing the HTM files
lyrics_directory = '/kaggle/input/lyrics-zip/'

# ## Step 4: Verify and list the files in the specified directory
print("\nFiles found in the specified directory:")
for filename in os.listdir(lyrics_directory):
    if filename.endswith('.htm') or filename.endswith('.html'):
        print(filename)

# ## Step 5: Function to extract lyrics and title from a single HTM file
def extract_lyrics_and_title_from_htm(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    japanese_lyrics = ""
    english_lyrics = ""
    title_tag = soup.find('title')
    title = title_tag.string.strip() if title_tag and title_tag.string else os.path.splitext(os.path.basename(file_path))[0]

    japanese_header = soup.find('p', string=lambda text: text and "Japanese" in text)
    if japanese_header and japanese_header.find('b'):
        next_element = japanese_header.find_next()
        while next_element and not (next_element.name == 'p' and next_element.find('b', string="English")):
            if next_element.name in ['p', 'div', 'span'] and next_element.get_text(strip=True):
                japanese_lyrics += next_element.get_text(strip=True) + "\n"
            next_element = next_element.find_next()

    english_header = soup.find('p', string=lambda text: text and "English" in text)
    if english_header and english_header.find('b'):
        next_element = english_header.find_next()
        while next_element and not (next_element.name == 'p' and next_element.find('b')):
            if next_element.name in ['p', 'div', 'span'] and next_element.get_text(strip=True):
                english_lyrics += next_element.get_text(strip=True) + "\n"
            next_element = next_element.find_next()

    japanese_lyrics = re.sub(r'\s+', ' ', japanese_lyrics).strip()
    english_lyrics = re.sub(r'\s+', ' ', english_lyrics).strip()

    return {
        'title': title,
        'japanese': japanese_lyrics,
        'english': english_lyrics,
        'filename': os.path.basename(file_path)
    }

# ## Step 6: Function to process all HTM files in a directory and capture titles
def process_htm_files_with_title(directory):
    all_lyrics_data = []
    for filename in os.listdir(directory):
        if filename.endswith('.htm') or filename.endswith('.html'):
            file_path = os.path.join(directory, filename)
            lyrics_data = extract_lyrics_and_title_from_htm(file_path)
            all_lyrics_data.append(lyrics_data)
    return all_lyrics_data

# ## Step 7: Extract lyrics and titles from all HTM files
all_lyrics_data = process_htm_files_with_title(lyrics_directory)
print(f"\nProcessed {len(all_lyrics_data)} lyric files and extracted titles.")

# ## Step 8: Configure the Gemini API and Google Search API
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-2.0-flash')
image_model = genai.GenerativeModel('gemini-2.0-flash')
translation_model = genai.GenerativeModel('gemini-2.0-flash')

GOOGLE_SEARCH_API_KEY = user_secrets.get_secret("GOOGLE_SEARCH_API_KEY")
GOOGLE_CSE_ID = user_secrets.get_secret("GOOGLE_CSE_ID")

print("Google API keys configured successfully.")

# ## Step 9: Function to analyze and summarize lyrics with title information (WITH CACHING AND JSON OPTION)
analysis_cache = {}

def analyze_and_summarize_lyrics(lyrics_data, model=None, output_format="text"):
    if not model:
        print("Error: Gemini model not provided for analysis.")
        return "Could not perform analysis."
    cache_key = (tuple(sorted([item['filename'] for item in lyrics_data])), output_format)
    if cache_key in analysis_cache:
        print(f"Using cached analysis (format: {output_format}).")
        return analysis_cache[cache_key]
    else:
        combined_lyrics = ""
        for data in lyrics_data:
            combined_lyrics += f"\n\nTitle: {data['title']}\nFile: {data['filename']}\n"
            combined_lyrics += f"Japanese Lyrics:\n{data['japanese']}\n"
            combined_lyrics += f"English Translation:\n{data['english']}\n"

        if output_format == "json":
            prompt = f"""
            You are an expert in analyzing bilingual song lyrics (Japanese and English).
            Analyze the following lyrics, paying attention to the song titles.
            Identify overall themes, overall sentiment, and for each song, its specific themes and sentiment.

            Provide your analysis as a JSON object with the following structure:
            {{
              "overall_analysis": {{
                "themes": ["theme1", "theme2", ...],
                "sentiment": "positive" | "negative" | "neutral" | "mixed"
              }},
              "song_analyses": [
                {{
                  "title": "Song Title 1",
                  "themes": ["themeA", "themeB"],
                  "sentiment": "positive" | "negative" | "neutral" | "mixed",
                  "key_insights": "..."
                }},
                {{
                  "title": "Song Title 2",
                  "themes": [...],
                  "sentiment": "...",
                  "key_insights": "..."
                }},
                ...
              ]
            }}

            Lyrics:\n{combined_lyrics}
            """
        else:  # output_format == "text"
            prompt = f"""
            You are an expert in analyzing bilingual song lyrics (Japanese and English).
            Analyze the following lyrics, paying attention to the song titles.
            Identify themes, sentiment in both languages, and key differences or similarities in expression.

            After your analysis, provide a concise summary of the main themes and
            overall sentiment across the collection of lyrics, referencing song titles where relevant.

            Lyrics:\n{combined_lyrics}
            """

        try:
            response = model.generate_content(prompt)
            result = response.text
            if output_format == "json":
                try:
                    import json
                    result = json.loads(response.text)
                except json.JSONDecodeError as e:
                    result = f"Error decoding JSON: {e}\nRaw response: {response.text}"
            analysis_cache[cache_key] = result
            return result
        except Exception as e:
            return f"Error during analysis and summarization: {e}"

# ## Step 10: Function to generate an image based on the lyric analysis summary
def generate_lyric_image(analysis_summary, model=None):
    if not model:
        print("Error: Gemini model not provided for image generation.")
        return "Could not generate image."
    image_prompt = f"""
    Generate a visually appealing and abstract image that represents the
    main themes and overall sentiment described in this summary of song lyrics:
    "{analysis_summary}"
    The image should evoke the general mood and concepts discussed, potentially
    drawing inspiration from song titles. Please return an actual image.
    """
    try:
        response = image_model.generate_content([image_prompt])
        print(f"Image Generation Response: {response}")
        if response.parts:
            part = response.parts[0]
            print(f"Response Part: {part}")
            if hasattr(part, 'mime_type') and hasattr(part, 'blob'):
                mime_type = part.mime_type
                data = part.blob.raw_bytes
                print(f"Found image data with mime type: {mime_type}")
                return mime_type, data
            elif hasattr(part, 'text'):
                print("Response part contains text:", part.text[:200])
                if part.text.startswith("data:image/"):
                    try:
                        mime_type, base64_data = part.text.split(';base64,')
                        mime_type = mime_type.split(':')[1]
                        image_bytes = base64.b64decode(base64_data)
                        print(f"Successfully decoded base64 image from text with mime type: {mime_type}")
                        return mime_type, image_bytes
                    except Exception as e:
                        print(f"Error decoding base64 image from text: {e}")
                        return "Could not decode image data from text."
                else:
                    return "Response part contains text, not image data."
            else:
                print("Response part has no mime_type, blob, or text attributes.")
                return "No usable data found in the response part."
        else:
            print("No response parts received.")
            return "No response parts received."
    except Exception as e:
        print(f"Error generating image: {e}")
        return f"Error generating image: {e}"

# ## Step 11: Function to translate text (WITH CACHING)
translation_cache = {}

def translate_a_line(text, target_language="English", model=None):
    if not model:
        print("Error: Gemini model not provided for translation.")
        return "Could not perform translation."
    cache_key = (text, target_language)
    if cache_key in translation_cache:
        print(f"Using cached translation for '{text}' to {target_language}.")
        return translation_cache[cache_key]
    else:
        translation_prompt = f"""
        Translate the following text to {target_language}:
        "{text}"
        """
        try:
            response = model.generate_content(translation_prompt)
            translation_cache[cache_key] = response.text
            return response.text
        except Exception as e:
            return f"Error during translation: {e}"

# ## Step 12: Function to answer freeform questions with title awareness
def answer_freeform_question(question, all_lyrics_data, analysis_summary=None, model=None):
    if not model:
        print("Error: Gemini model not configured.")
        return "I cannot answer your question at this time."

    context = ""
    if all_lyrics_data:
        context += "Here are the lyrics I have analyzed:\n"
        for data in all_lyrics_data:
            context += f"Title: {data['title']}\nFile: {data['filename']}\nJapanese: {data['japanese'][:100]}...\nEnglish: {data['english'][:100]}...\n\n"
    if analysis_summary:
        context += f"\nHere is the summary of the lyrics:\n{analysis_summary}\n\n"

    prompt = f"""You have access to the following information about song lyrics, including their titles:
    {context}

    Please answer the following question based on this information, referencing song titles if relevant:
    {question}

    If the information is not directly available, try to infer or provide a helpful response based on your understanding of the lyrics and the summary. If you cannot answer, please say so.
    """

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Error answering question: {e}"

# ## Step 13: Enhanced Interactive Agent Function with Title Awareness (WITH CACHING AND JSON OPTION)
def interactive_agent(all_lyrics, last_analysis_summary=None, model=None, image_model=None, translation_model=None, google_api_key=None, google_cse_id=None):
    print("\n--- Interactive Lyric Agent ---")
    print("You can ask me freeform questions about the lyrics (including song titles) or use the following commands:")
    print("'analyze' - Analyze all lyrics and provide a text summary.")
    print("'analyze --json' - Analyze all lyrics and provide a JSON output.")
    print("'draw' - Generate an image based on the last analysis (evaluates visually).")
    print("'translate <text> [to <language>]' - Translate text (evaluates by manual inspection).")
    print("'translate_ja <filename>' - Translate all Japanese lyrics of a specific file to English.")
    print("'find <keyword>' - Find occurrences of a keyword in the lyrics and titles.")
    print("'info <filename>' - Search Google for more information about the song (using filename to get title).")
    print("'info title <song_title>' - Search Google for more information about a specific song title.")
    print("'exit' - Exit the agent.")
    print("\nJust type your question directly to ask about the lyrics.")

    while True:
        user_input = input("> ")
        user_input_lower = user_input.lower()

        if user_input_lower == 'exit':
            print("Exiting the lyric agent.")
            break
        elif user_input_lower == 'analyze':
            if len(user_input.split()) > 1 and user_input.split()[1] == '--json':
                print("\nInitiating analysis of all lyrics (JSON format)...")
                analysis_json = analyze_and_summarize_lyrics(all_lyrics, model=model, output_format="json")
                print("\nAnalysis (JSON):")
                import json
                if isinstance(analysis_json, str) and analysis_json.startswith("Error decoding JSON"):
                    print(analysis_json)
                else:
                    print(json.dumps(analysis_json, indent=2))
                print("\n--- Evaluation of Analysis (JSON) ---")
                print("Please review the structure and content of the JSON output.")
                last_analysis_summary = "JSON Analysis Performed: See output above." # Update summary type
            else:
                print("\nInitiating analysis of all lyrics (text format)...")
                analysis = analyze_and_summarize_lyrics(all_lyrics, model=model, output_format="text")
                print("\nAnalysis (Text):")
                print(analysis)
                print("\n--- Evaluation of Analysis (Text) ---")
                print("Please review the analysis for coherence, relevance, completeness, and accuracy, considering the song titles.")
                last_analysis_summary = analysis
        elif user_input_lower == 'draw':
            if last_analysis_summary and not last_analysis_summary.startswith("JSON Analysis Performed"):
                print("\nGenerating an image based on the last text analysis summary...")
                image_output = generate_lyric_image(last_analysis_summary, model=image_model)
                if isinstance(image_output, tuple):
                    mime_type, data = image_output
                    print(f"\nImage generated with mime type: {mime_type}")
                    try:
                        if mime_type.startswith('image/'):
                            image = Image.open(io.BytesIO(data))
                            print("\nGenerated Image:")
                            display(image)
                            print("\n--- Visual Evaluation ---")
                            print("Does the image visually represent the themes and sentiment of the lyrics summary, potentially reflecting song titles?")
                        else:
                            print("Generated data is not an image.")
                    except Exception as e:
                        print(f"Error displaying image: {e}")
                else:
                    print(f"Image generation failed: {image_output}")
            elif last_analysis_summary and last_analysis_summary.startswith("JSON Analysis Performed"):
                print("Cannot generate an image directly from JSON analysis. Please run 'analyze' without the '--json' option first.")
            else:
                print("Please run 'analyze' first to generate a summary before drawing.")
        elif user_input_lower.startswith('translate_ja'):
            parts = user_input.split()
            if len(parts) == 2:
                filename_to_translate = parts[1]
                found_lyrics = next((item for item in all_lyrics if item['filename'].lower() == filename_to_translate.lower()), None)
                if found_lyrics and found_lyrics['japanese']:
                    print(f"\nTranslating Japanese lyrics of '{found_lyrics['title']}' to English...")
                    japanese_lines = found_lyrics['japanese'].strip().split('\n')
                    english_translations = []
                    for line in japanese_lines:
                        translation = translate_a_line(line, target_language="English", model=translation_model)
                        english_translations.append(translation)
                    print("\n--- English Translation of Japanese Lyrics ---")
                    for i, translation in enumerate(english_translations):
                        print(f"{japanese_lines[i]}\n> {translation}\n")
                    print("\n--- Translation Evaluation ---")
                    print("Please evaluate the accuracy and fluency of these translations.")
                elif not found_lyrics:
                    print(f"Error: Lyric file '{filename_to_translate}' not found.")
                elif not found_lyrics['japanese']:
                    print(f"Error: No Japanese lyrics found in '{found_lyrics['title']}'.")
            else:
                print("Usage: 'translate_ja <filename>' (e.g., 'translate_ja song1.htm')")
        elif user_input_lower.startswith('translate'):
            parts = user_input.split()
            if len(parts) >= 2:
                text_to_translate = ' '.join(parts[1:])
                target_language = "English"
                if "to" in parts:
                    to_index = parts.index("to")
                    if to_index + 1 < len(parts):
                        target_language = ' '.join(parts[to_index + 1:]).capitalize()
                        text_to_translate = ' '.join(parts[1:to_index])
                print(f"\nTranslating '{text_to_translate}' to {target_language}...")
                translation = translate_a_line(text_to_translate, target_language, model=translation_model)
                print(f"Translation: {translation}")
                print("\n--- Translation Evaluation ---")
                print(f"Please evaluate the translation for accuracy and fluency in {target_language}.")
            else:
                print("Please provide text to translate. Example: 'translate konnichiwa'")
        elif user_input_lower.startswith('find'):
            parts = user_input.split()
            if len(parts) >= 2:
                keyword = parts[1].lower()
                found_in = {}
                for lyric_data in all_lyrics:
                    if keyword in lyric_data['japanese'].lower() or keyword in lyric_data['english'].lower() or keyword in lyric_data['title'].lower():
                        found_in[lyric_data['filename']] = []
                        if keyword in lyric_data['title'].lower():
                            found_in[lyric_data['filename']].append(f"(Title) {lyric_data['title']}")
                        if keyword in lyric_data['japanese'].lower():
                            for line in lyric_data['japanese'].split('\n'):
                                if keyword in line.lower():
                                    found_in[lyric_data['filename']].append(f"(Japanese) {line}")
                        if keyword in lyric_data['english'].lower():
                            for line in lyric_data['english'].split('\n'):
                                if keyword in line.lower():
                                    found_in[lyric_data['filename']].append(f"(English) {line}")

                if found_in:
                    print(f"\nKeyword '{keyword}' found in the following lyrics and titles:")
                    for filename, lines in found_in.items():
                        print(f"- {filename}:")
                        for line in lines:
                            print(f"  - {line}")
                else:
                    print(f"\nKeyword '{keyword}' not found in any of the lyrics or titles.")
            else:
                print("Please provide a keyword to find. Example: 'find love'")
        elif user_input_lower.startswith('info'):
            parts = user_input.split()
            if len(parts) >= 2:
                if parts[1] == 'title' and len(parts) >= 3:
                    song_title = ' '.join(parts[2:])
                    search_query = song_title
                    print(f"\nSearching Google for title: '{search_query}'...")
                    search_results = search_google(search_query, google_api_key, google_cse_id)
                    if search_results:
                        print("\n--- Google Search Results for Title ---")
                        for i, result in enumerate(search_results):
                            print(f"{i+1}. Title: {result.get('title')}")
                            print(f"   Link: {result.get('link')}")
                            snippet = result.get('snippet')
                            if snippet:
                                print(f"   Snippet: {snippet}")
                            print("-" * 20)
                    else:
                        print("No relevant search results found for this title.")
                elif len(parts) == 2:
                    filename_to_search = parts[1]
                    found_lyrics = next((item for item in all_lyrics if item['filename'].lower() == filename_to_search.lower()), None)
                    if found_lyrics:
                        song_title = found_lyrics.get('title', os.path.splitext(filename_to_search)[0])
                        search_query = song_title
                        print(f"\nSearching Google for title from filename '{filename_to_search}': '{search_query}'...")
                        search_results = search_google(search_query, google_api_key, google_cse_id)
                        if search_results:
                            print("\n--- Google Search Results for Title ---")
                            for i, result in enumerate(search_results):
                                print(f"{i+1}. Title: {result.get('title')}")
                                print(f"   Link: {result.get('link')}")
                                snippet = result.get('snippet')
                                if snippet:
                                    print(f"   Snippet: {snippet}")
                                print("-" * 20)
                        else:
                            print("No relevant search results found for this title.")
                    else:
                        print(f"Error: Lyric file '{filename_to_search}' not found.")
                else:
                    print("Usage: 'info <filename>' or 'info title <song_title>'")
            else:
                # Treat any other input as a freeform question
                print("\nThinking...")
                answer = answer_freeform_question(user_input, all_lyrics, last_analysis_summary, model=model)
                print(f"Agent: {answer}")

# ## Step 14: Perform initial analysis (without initial image generation)
print("\nInitiating analysis and summarization with Gemini-2.0-Flash (including titles)...")
initial_analysis_summary = analyze_and_summarize_lyrics(all_lyrics_data, model=model)
print("\nAnalysis and Summary complete. Gemini-2.0-Flash's response:")
print(initial_analysis_summary)

# ## Step 15: Example of Japanese to English translation
if all_lyrics_data and all_lyrics_data[0]:
    first_song = all_lyrics_data[0]
    print(f"\nExample Song Title: {first_song.get('title', 'No Title Found')}")

    # Show Japanese lyrics
    if first_song.get('japanese'):
        print("\n--- Original Japanese Lyrics ---")
        print(first_song['japanese'].strip())

        # Translate the entire Japanese lyrics to English
        print(f"\nTranslating all Japanese lyrics of '{first_song['title']}' to English...")
        japanese_lyrics = first_song['japanese'].strip()
        english_translation_gemini = translate_a_line(japanese_lyrics, target_language="English", model=translation_model)
        print("\n--- English Translation of Japanese Lyrics (via Gemini) ---")
        print(english_translation_gemini)
        print("\n--- Translation Evaluation ---")
        print("Please evaluate the accuracy and fluency of this English translation.")
    else:
        print("\nNo Japanese lyrics found for this song.")

else:
    print("\nNo lyrics data available to demonstrate translation.")

# ## Step 16: Start the interactive agent
interactive_agent(all_lyrics_data, initial_analysis_summary, model=model, image_model=image_model, translation_model=translation_model, google_api_key=GOOGLE_SEARCH_API_KEY, google_cse_id=GOOGLE_CSE_ID)

Installed google-generativeai
Installed pillow
Installed beautifulsoup4
Installed google-generativeai
Installed google-api-python-client

Files found in the specified directory:
kiraku.htm
anatanosei.htm
konna_ni.htm
mou_sukoshi.htm
kimi_ga_ita.htm
tooi_hi.htm
another_day.htm
nemurenai.htm
i_want_you.htm
nemurezu_ni.htm
yureru.htm
hero.htm
forever_you.htm
ienakute.htm
sukidakedo.htm
kononamida.htm
hiraite.htm
loneliness.htm
dontyousee.htm
in_my_arms.htm
sorasanaide.htm
sekaiwakitto.htm
just_believe.htm
ameni_nurete.htm
mezametaasa.htm
so_together.htm
itaikurai.htm
watashidake.htm
iki_mo.htm
monochrome.htm
nemuri.htm
boy.htm
anatani.htm
readygo.htm
geturedream.htm
fushigine.htm
theonlytruth.htm
anohohoemi.htm
nemutteru.htm
im_in_love.htm
hateshinai.htm
oyogi.htm
season.htm
aitaku.htm
door.htm
new_love.htm
promisedyou.htm
kaeranutoki.htm
wake_up.htm
matteru.htm
kimi_ga_inai.htm
dangerous.htm
leave_me.htm
sukina_youni.htm
dan_dan.htm
kanariya.htm
straylove.htm
toori_nukeru.htm
kanjiteitai

KeyboardInterrupt: Interrupted by user